In [3]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message,ibConnection
import ib.opt
import ib.ext.Contract
from ib.ext.TickType import TickType as tt
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [18]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 5
        self.rcdId = 1000
        self.updated=0

        self.con = ibConnection(**kwargs)
        
        self.con.registerAll(self.watcher)
        self.contracts = {}
        self.prices = pd.DataFrame(columns = ['id','bidPrice','askPrice'])
        self.lastTime = pd.DataFrame(columns = ['id','bidPrice','askPrice'])


        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler,
            ib.opt.message.tickPrice: self.tickPriceHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()
        time.sleep(1)
        self.con.reqMarketDataType( 3 )
    
    def addContract(self,contract):
        n= self.prices.shape[0]
        id = 100+n
        self.contracts[id] = contract
        self.prices.loc[n] = pd.Series({'id':id,'bidPrice':0,'askPrice':0})
        self.lastTime.loc[n] = pd.Series({'id':id,'bidPrice':0,'askPrice':0})
    
    def tickPriceHandler(self, msg):
        #print (msg,msg.tickerId,msg.field,msg.price)
        if msg.field in [66,67]:
            self.con.cancelMktData(msg.tickerId)
            self.updated=self.updated+1
            if (self.updated>=2*self.prices.shape[0]):
                print ('Elapsed: %s' % (time.time() - self.t))
            print (msg.tickerId,msg.field,msg.price)
            if msg.field == 66:
                field = 'bidPrice'
                
                if 'frontOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.frontOptStruct.index:
                        strike = self.frontOptStruct.loc[msg.tickerId].m_strike
                        right = self.frontOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.frontOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.frontOptPrices.loc[strike,'CALL BID'] = msg.price
                elif 'secondOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.secondOptStruct.index:
                        strike = self.secondOptStruct.loc[msg.tickerId].m_strike
                        right = self.secondOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.secondOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.secondOptPrices.loc[strike,'CALL BID'] = msg.price
                elif 'esOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.esOptStruct.index:
                        strike = self.esOptStruct.loc[msg.tickerId].m_strike
                        right = self.esOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.esOptPrices.loc[strike,'PUT BID'] = msg.price
                        elif right=='C':
                            self.esOptPrices.loc[strike,'CALL BID'] = msg.price
                
            if msg.field == 67:
                field = 'askPrice'  
                
                if 'frontOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.frontOptStruct.index:
                        strike = self.frontOptStruct.loc[msg.tickerId].m_strike
                        right = self.frontOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.frontOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.frontOptPrices.loc[strike,'CALL ASK'] = msg.price
                elif 'secondOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.secondOptStruct.index:
                        strike = self.secondOptStruct.loc[msg.tickerId].m_strike
                        right = self.secondOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.secondOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.secondOptPrices.loc[strike,'CALL ASK'] = msg.price
                elif 'esOptStruct' in list(self.__dict__.keys()):
                    if msg.tickerId in self.esOptStruct.index:
                        strike = self.esOptStruct.loc[msg.tickerId].m_strike
                        right = self.esOptStruct.loc[msg.tickerId].m_right

                        if right=='P':
                            self.esOptPrices.loc[strike,'PUT ASK'] = msg.price
                        elif right=='C':
                            self.esOptPrices.loc[strike,'CALL ASK'] = msg.price
                        
            self.prices.loc[self.prices.id==msg.tickerId,field] = msg.price

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))
            
    def get_market_data(self):
        print ("Calling Market Data")
        self.t = time.time()
        self.updated=0
        for key in self.contracts.keys():          
            self.con.reqMktData(key, self.contracts[key], "66,67", True)
            time.sleep(0.01)

    def get_contract_details(self, contract):
        self.rcdId=self.rcdId+1
        self.con.reqContractDetails(self.rcdId, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            try:
                if df.shape[0]==0:
                    df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                    df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
                else:
                    df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            except:
                pass

        return df, err, mid, msg

In [19]:
def vixFutContract(localSym=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'
    contract.m_localSymbol = localSym

    return contract

def spFutContract(localSymbol=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'SP'
    contract.m_localSymbol = localSymbol
    
    return contract

def esFutContract(localSymbol=None):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    contract.m_localSymbol = localSymbol
    
    return contract

def vxOptContract(exp, localSymbol=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    contract.m_localSymbol = localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='VIX'
 
    return contract

def spOptContract(exp=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    contract.m_expiry = exp
    #contract.m_tradingClass='SP'
 
    return contract

def esOptContract(exp=None, localSymbol=None):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    contract.m_expiry = exp
    contract.m_tradingClass="ES"
    contract.m_localSymbol = localSymbol
 
    return contract

In [21]:
#ibm.con.disconnect()
ibm = IbManager("127.0.0.1", port=7497, clientId=999)

Server Version: 76
TWS Time at connection:20181219 17:29:07 EST
---------- <managedAccounts accountsList=DU1300822>
---------- <nextValidId orderId=2>
---------- <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture.nj>
---------- <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm.nj>
---------- <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
---------- <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
---------- <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
---------- <error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:njhmds>
---------- <error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ushmds>


In [22]:
#read config
cfg = pd.read_json('config.json')
cfg

,name,optionsExp,symbol
0,second,20190521,VXG9
1,front,20190115,VXF9
2,sp,20190117,SPH9


In [25]:
ibm.addContract(vixFutContract(cfg[cfg.name=='front'].symbol.values[0]))
ibm.addContract(vixFutContract(cfg[cfg.name=='second'].symbol.values[0]))
ibm.addContract(spFutContract(localSymbol=cfg[cfg.name=='sp'].symbol.values[0]))

In [29]:
#create dataFrame for VIX options
optCtrct = vxOptContract(str(cfg[cfg.name=='front'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.frontOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.frontOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = vxOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [538]:
#create dataFrame for VIX options
optCtrct = vxOptContract(str(cfg[cfg.name=='second'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.secondOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.secondOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = vxOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [451]:
#create dataFrame for ES options
optCtrct = esOptContract(str(cfg[cfg.name=='es'].optionsExp.values[0]))
cdetails, err, errid, errmsg = ibm.get_contract_details(optCtrct)
cdetails=cdetails.sort_values('m_strike')

cdetails['id']=[100+ibm.prices.shape[0]+x for x in (range(cdetails.shape[0]))]
ibm.esOptPrices = pd.DataFrame(index=set(cdetails.m_strike),columns=['PUT BID','PUT ASK','CALL BID','CALL ASK']).sort_index()
ibm.esOptStruct = cdetails[['id','m_strike','m_right']].set_index('id').sort_index()

for i,row in cdetails.iterrows():
    ctrct = esOptContract(row.m_expiry, row.m_localSymbol)
    ibm.addContract(ctrct)

In [27]:
ibm.get_market_data()

Calling Market Data
---------- <marketDataType reqId=100, marketDataType=3>
100 66 22.25
100 67 22.4
---------- <tickString tickerId=100, tickType=88, value=1545256798>
---------- <tickSnapshotEnd reqId=100>
---------- <marketDataType reqId=103, marketDataType=3>
103 66 22.25
103 67 22.4
---------- <tickString tickerId=103, tickType=88, value=1545256798>
---------- <tickSnapshotEnd reqId=103>
---------- <marketDataType reqId=101, marketDataType=3>
---------- <marketDataType reqId=104, marketDataType=3>
---------- <error id=101, errorCode=10167, errorMsg=Requested market data is not subscribed. Displaying delayed market data...>
---------- <marketDataType reqId=102, marketDataType=3>
---------- <marketDataType reqId=105, marketDataType=3>
---------- <error id=102, errorCode=10167, errorMsg=Requested market data is not subscribed. Displaying delayed market data...>
102 66 -1.0
102 67 -1.0
105 66 -1.0
105 67 -1.0
---------- <tickString tickerId=102, tickType=88, value=1545256659>
--------

In [31]:
ibm.con.reqMktData(21, ctrct, "66,67", True)

---------- <marketDataType reqId=21, marketDataType=3>
---------- <error id=21, errorCode=10167, errorMsg=Requested market data is not subscribed. Displaying delayed market data...>
---------- <tickOptionComputation tickerId=21, field=83, impliedVol=1.3819861339814359, delta=0.0025771680358810584, optPrice=0.00592167852335457, pvDividend=0.0, gamma=0.0009340504131486395, vega=0.0004927698069670962, theta=-0.00121735131097228, undPrice=22.32293343266863>
21 66 -1.0
21 67 -1.0
---------- <tickString tickerId=21, tickType=88, value=1545239646>
---------- <tickSize tickerId=21, field=74, size=6>
---------- <error id=-1, errorCode=2108, errorMsg=Market data farm connection is inactive but should be available upon demand.usopt>
---------- <error id=-1, errorCode=2108, errorMsg=Market data farm connection is inactive but should be available upon demand.usopt>
---------- <error id=-1, errorCode=2108, errorMsg=Market data farm connection is inactive but should be available upon demand.usfuture.

19-Dec-18 23:14:04 ERROR     Exception in message dispatch.  Handler 'watcher' for 'error'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/ib/ext/EReader.py", line 107, in run
    while not self.isInterrupted() and self.processMsg(self.readInt()):
  File "/opt/conda/lib/python3.7/site-packages/ib/ext/EReader.py", line 910, in readInt
    strval = self.readStr()
  File "/opt/conda/lib/python3.7/site-packages/ib/ext/EReader.py", line 895, in readStr
    c = self.m_dis.readByte()
  File "/opt/conda/lib/python3.7/site-packages/ib/lib/__init__.py", line 125, in readByte
    return unpack('!b', self.recv(1))[0]
struct.error: unpack requires a buffer of 1 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/ib/opt/dispatcher.py", line 44, in __call__
    results.append(listener(message))
  File "<ipython-input-18-cf891a4b7167>", line 113, in watcher
    if

---------- <connectionClosed>
